### Используем НОРМАЛЬНЫЙ апи клиент, который по хорошему надо бы интегрировать в дансвер

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
NOTION_API_KEY = os.getenv("NOTION_API_KEY")
NOTION_ROOT_ID = os.getenv("NOTION_ROOT_ID")
NOTION_DATABASE_ID = os.getenv("NOTION_DATABASE_ID")

In [ ]:
from notion_client import Client, AsyncClient

# Initialize a client with the integration token
notion = Client(auth=NOTION_API_KEY)


In [ ]:
page_id = NOTION_ROOT_ID

# Retrieve the page
page = notion.pages.retrieve(page_id=page_id)

# Print page properties
print(page["properties"])


In [ ]:

pages = notion.databases.query(database_id=NOTION_DATABASE_ID) # корретно влзвращает и страницы и теги, но нужно не проморгать пагинацию (has_more и next_cursor)

In [ ]:
page = pages['results'][1]

In [ ]:
page['properties']['Tags']['multi_select']

In [ ]:
notion.pages.retrieve(page_id=database_id)

In [ ]:
has_more : bool = pages["has_more"]
cursor : str = pages["next_cursor"]

In [ ]:
pages['next_cursor']

In [ ]:
database_id = NOTION_DATABASE_ID
notion.databases.retrieve(database_id=database_id)

In [ ]:
def retrieve_blocks(page_id):
    all_blocks = []
    start_cursor = None
    while True:
        response = notion.blocks.children.list(block_id=page_id, start_cursor=start_cursor)
        blocks = response.get("results", [])
        all_blocks.extend(blocks)
        if not response.get("has_more", False):
            break
        start_cursor = response.get("next_cursor", None)
    return all_blocks

root_page_id = "a283ff3a46804b39b50e29c566369c2f"

# Retrieve blocks from the root page
blocks = retrieve_blocks(root_page_id)
visited_ids = set()

def retrieve_all_blocks(blocks, level=0):
    all_blocks = []
    for block in blocks:
        if block["id"] not in visited_ids:
            #print("  " * level + block["type"], block.get("id", ""))
            all_blocks.append(block)
            visited_ids.add(block["id"])
            if block.get("has_children", False):
                child_blocks = retrieve_blocks(block["id"])
                new_blocks = retrieve_all_blocks(child_blocks, level + 1)
                all_blocks.extend(new_blocks) if new_blocks else None
    return all_blocks

# Start retrieving all nested blocks
all_blocks = retrieve_all_blocks(blocks)


In [ ]:
for i, block in enumerate(all_blocks):
    if "https://www.defenseurdesdroits.fr/carte-des-delegues" in str(block):
        print(block)
        print(i)


In [ ]:
print(all_blocks[131])

### Используем то что уже зашито в дасвер, чтобы ничего не сломать

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
NOTION_API_KEY = os.getenv("NOTION_API_KEY")
NOTION_ROOT_ID = os.getenv("NOTION_ROOT_ID")
NOTION_DATABASE_ID = os.getenv("NOTION_DATABASE_ID")

In [ ]:
from danswer.connectors.notion.connector import NotionConnector

In [ ]:
connector = NotionConnector(root_page_id="f2c93c208bd345038ef7e99f9db7b23e") # just one page here
#connector = NotionConnector(root_page_id=NOTION_DATABASE_ID) # database with pages and tags
connector.load_credentials({"notion_integration_token": NOTION_API_KEY})
connector.is_db

In [ ]:
out = [_ for _ in connector.load_from_state()]

In [ ]:
pages = [connector._fetch_page("a283ff3a46804b39b50e29c566369c2f")]

In [ ]:
parsed_pages = []
for parsed_page in connector._read_pages(pages):
    parsed_pages.append(parsed_page)

In [ ]:
for section in parsed_pages[0]:
    print(section)